In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#formatting the text
data = open('data/Othello.txt').read().lower()
chars = sorted(list(set(data)))
total_chars = len(data)
unique_chars = len(chars)

In [3]:
chars_for_ids = {char:Id for Id, char in enumerate(chars)}
ids_for_chars = {Id:char for Id, char in enumerate(chars)}
chars_to_learn = 100

In [4]:
char_X = []
y = []
counter = total_chars - chars_to_learn
for i in range(0, counter, 1):
    input_chars = data[i:i+chars_to_learn]
    output_chars = data[i + chars_to_learn]
    char_X.append([chars_for_ids[char] for char in input_chars])
    y.append(chars_for_ids[output_chars])
    

In [5]:
X = np.reshape(char_X, (len(char_X), chars_to_learn, 1))
X = X/float(unique_chars)
y = np_utils.to_categorical(y)

In [6]:
#creating and fitting the model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=128)


Epoch 1/20
1760/1760 [==============================] - 13s 7ms/step - loss: 3.2549
Epoch 2/20
1760/1760 [==============================] - 13s 7ms/step - loss: 3.0352
Epoch 3/20
1760/1760 [==============================] - 12s 7ms/step - loss: 3.0093
Epoch 4/20
1760/1760 [==============================] - 12s 7ms/step - loss: 2.9979
Epoch 5/20
1760/1760 [==============================] - 12s 7ms/step - loss: 2.9933
Epoch 6/20
1760/1760 [==============================] - 12s 7ms/step - loss: 2.9929
Epoch 7/20
1760/1760 [==============================] - 13s 7ms/step - loss: 2.9914
Epoch 8/20
1760/1760 [==============================] - 12s 7ms/step - loss: 2.9915
Epoch 9/20
1760/1760 [==============================] - 12s 7ms/step - loss: 2.9889
Epoch 10/20
1760/1760 [==============================] - 12s 7ms/step - loss: 2.9864
Epoch 11/20
1760/1760 [==============================] - 12s 7ms/step - loss: 2.9903
Epoch 12/20
1760/1760 [==============================] - 12s 7ms/step - lo

In [7]:
import h5py
model.save_weights('data/Othello.hdf5')

In [8]:
model.load_weights('data/Othello.hdf5')

In [14]:
random_val = np.random.randint(0, len(char_X)-1)
random_start = char_X[random_val]
for i in range(500):
    x = np.reshape(random_start, (1, len(random_start), 1))
    x = x/float(unique_chars)
    pred = model.predict(x)
    index = np.argmax(pred)
    random_start.append(index)
    random_start = random_start[1: len(random_start)]
print("".join([ids_for_chars[value] for value in random_start]))

In [31]:
from random import randint
def make_seed():
    sentence_length=100
    seed_phrase="i love you"
    if seed_phrase:
        phrase_length = len(seed_phrase)
        pattern = ""
        for i in range (0, sentence_length):
            pattern += seed_phrase[i % phrase_length]
    else:
        seed = randint(0, len(data) - sentence_length)
        pattern = data[seed:seed + sentence_length]

    return pattern

In [34]:
X = np.zeros((1, 100, len(chars)), dtype=np.bool)
seed = 7
seed_pattern = make_seed()
for i, character in enumerate(seed_pattern):
    X[0, i, ids_for_chars[character]] = 1
generated_text = ""
for i in range(500):
    prediction = np.argmax(model.predict(X, verbose=0))
    
    generated_text += chars_for_ids[prediction]
#    activations = np.zeros((1, 1, len(chars)), dtype=np.bool)
#    activations[0, 0, prediction] = 1
#    X = np.concatenate((X[:, 1:, :], activations), axis=1)
print(generated_text)

KeyError: 'i'